In [34]:
# Data Handling and Processing
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from dotenv import dotenv_values

# Machine Learning Models and Evaluation Metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Visualization (Optional, if you'd like to visualize predictions or residuals)
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress Warnings
import warnings
warnings.filterwarnings('ignore')


In [35]:
# Load environment variables
config = dotenv_values()

# Define variables for the login
pg_user = config['POSTGRES_USER']
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']
pg_pass = config['POSTGRES_PASS']

# Set up the PostgreSQL connection URL
url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'

# Create the database engine
engine = create_engine(url, echo=False)
my_schema = 'capstone_barstov_industries'

# Load data directly into a DataFrame
with engine.connect() as conn:
    conn.execute(text(f'SET search_path TO {my_schema};'))
    data = pd.read_sql("SELECT * FROM model_data_week;", conn)

# Check the DataFrame structure
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172977 entries, 0 to 172976
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   product_type_no    172977 non-null  int64  
 1   product_type_name  172977 non-null  object 
 2   colour_group_code  172977 non-null  int64  
 3   colour_group_name  172977 non-null  object 
 4   week               172977 non-null  object 
 5   average_price      172977 non-null  float64
 6   total_units_sold   172977 non-null  int64  
 7   unique_customers   172977 non-null  int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 10.6+ MB


In [37]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Ensure 'week' is in datetime format without timezone
data['week'] = pd.to_datetime(data['week']).dt.tz_localize(None)

# Filter data up to March 2020 and sort
data = data[data['week'] <= "2020-03-01"].copy()
data = data.sort_values(by=['product_type_no', 'colour_group_code', 'week'])

# Step 1: Data-Driven Peak Season Identification Using Quantile-Based Thresholds

# Calculate 90th percentile threshold for peak season within each product-type and color group
demand_clusters = data.groupby(['product_type_no', 'colour_group_code', 'week'])['total_units_sold'].sum().reset_index()
demand_clusters['peak_threshold'] = demand_clusters.groupby(['product_type_no', 'colour_group_code'])['total_units_sold'].transform(lambda x: x.quantile(0.90))

# Flag as peak season if total_units_sold exceeds the 90th percentile threshold
demand_clusters['is_peak_season'] = demand_clusters['total_units_sold'] >= demand_clusters['peak_threshold']

# Merge peak season flags back into main dataset
data = pd.merge(data, demand_clusters[['product_type_no', 'colour_group_code', 'week', 'is_peak_season']], 
                on=['product_type_no', 'colour_group_code', 'week'], how='left')

# Remove duplicate columns if any
if 'is_peak_season_x' in data.columns and 'is_peak_season_y' in data.columns:
    data = data.drop(columns=['is_peak_season_x']).rename(columns={'is_peak_season_y': 'is_peak_season'})

# Step 2: Feature Engineering - Lagged Features and Seasonal Features

data['lag_units_sold_1week'] = data.groupby(['product_type_no', 'colour_group_code'])['total_units_sold'].shift(1)
data['lag_units_sold_2weeks'] = data.groupby(['product_type_no', 'colour_group_code'])['total_units_sold'].shift(2)
data = data.dropna(subset=['lag_units_sold_1week', 'lag_units_sold_2weeks']).reset_index(drop=True)

# Month feature for seasonality
data['month'] = data['week'].dt.month

# Scale the price
data['average_price'] = data['average_price'] * 10


# Reapply the train, validation, and test splits
train_data = data[(data['week'] >= "2019-01-01") & (data['week'] < "2020-01-01")]
validation_data = data[(data['week'] >= "2020-01-01") & (data['week'] < "2020-02-01")]
test_data = data[(data['week'] >= "2020-02-01") & (data['week'] < "2020-03-01")]

# Print sizes for verification
print("Training set size:", train_data.shape)
print("Validation set size:", validation_data.shape)
print("Test set size:", test_data.shape)


# Step 4: Include `is_peak_season` in the features and proceed with model training

# Define feature columns
feature_columns = [
    'product_type_no', 'colour_group_code', 'average_price',
    'lag_units_sold_1week', 'lag_units_sold_2weeks', 'month', 'is_peak_season'
]

X_train, y_train = train_data[feature_columns], train_data['total_units_sold']
X_validation, y_validation = validation_data[feature_columns], validation_data['total_units_sold']
X_test, y_test = test_data[feature_columns], test_data['total_units_sold']

# Initialize and train base models
model_rf = RandomForestRegressor(n_estimators=200, max_depth=30, random_state=42)
model_lr = LinearRegression()
model_mlp = MLPRegressor(hidden_layer_sizes=(100,), max_iter=500, random_state=42)

model_rf.fit(X_train, y_train)
model_lr.fit(X_train, y_train)
model_mlp.fit(X_train, y_train)

# Generate predictions and round to integers
preds_rf_val = np.round(model_rf.predict(X_validation)).astype(int)
preds_lr_val = np.round(model_lr.predict(X_validation)).astype(int)
preds_mlp_val = np.round(model_mlp.predict(X_validation)).astype(int)

preds_rf_test = np.round(model_rf.predict(X_test)).astype(int)
preds_lr_test = np.round(model_lr.predict(X_test)).astype(int)
preds_mlp_test = np.round(model_mlp.predict(X_test)).astype(int)

# Meta-model predictions
val_predictions_df = pd.DataFrame({'RandomForest': preds_rf_val, 'LinearRegression': preds_lr_val, 'MLPRegressor': preds_mlp_val})
test_predictions_df = pd.DataFrame({'RandomForest': preds_rf_test, 'LinearRegression': preds_lr_test, 'MLPRegressor': preds_mlp_test})

final_model_rf = RandomForestRegressor(n_estimators=200, max_depth=30, random_state=42)
final_model_rf.fit(val_predictions_df, y_validation)

# Generate and round meta-model predictions
final_preds_val = np.round(final_model_rf.predict(val_predictions_df)).astype(int)
final_preds_test = np.round(final_model_rf.predict(test_predictions_df)).astype(int)

# Calculate metrics
mae_val = mean_absolute_error(y_validation, final_preds_val)
rmse_val = mean_squared_error(y_validation, final_preds_val, squared=False)
mae_test = mean_absolute_error(y_test, final_preds_test)
rmse_test = mean_squared_error(y_test, final_preds_test, squared=False)

print("Extended Validation MAE:", mae_val)
print("Extended Validation RMSE:", rmse_val)
print("Test MAE:", mae_test)
print("Test RMSE:", rmse_test)

# Aggregate inventory needs by product type and color for the test period
test_data['predicted_inventory_needs'] = final_preds_test
inventory_needs = test_data.groupby(['product_type_no', 'colour_group_code'])['predicted_inventory_needs'].sum().reset_index()

# Calculate average demand for each product-type and color combination
avg_demand = data.groupby(['product_type_no', 'colour_group_code'])['total_units_sold'].mean().reset_index()
avg_demand = avg_demand.rename(columns={'total_units_sold': 'avg_demand'})

# Merge average demand back into `test_data`
test_data = pd.merge(test_data, avg_demand, on=['product_type_no', 'colour_group_code'], how='left')
# Define multipliers for peak and non-peak seasons
# Prioritize rebalancing non-peak season items by setting a stricter (lower) threshold
non_peak_season_multiplier = 0.5  # Trigger rebalancing sooner for non-peak items (50% of average demand)
peak_season_multiplier = 1.0      # Allow peak items more space (100% of average demand)

# Calculate the low-demand threshold dynamically based on the seasonality of each item
test_data['low_demand_threshold'] = np.where(
    test_data['is_peak_season'],
    test_data['avg_demand'] * peak_season_multiplier,     # Use peak multiplier for peak season items
    test_data['avg_demand'] * non_peak_season_multiplier  # Use non-peak multiplier for non-peak items
)

# Calculate a rolling 4-week sum of predicted inventory needs to identify low-demand periods
test_data['low_demand_4weeks'] = test_data.groupby(['product_type_no', 'colour_group_code'])['predicted_inventory_needs'] \
    .transform(lambda x: x.rolling(window=4, min_periods=1).sum())

# Flag items for rebalancing based on the dynamic low-demand threshold
test_data['rebalance_flag'] = test_data['low_demand_4weeks'] <= test_data['low_demand_threshold']

# Summarize the flagged items for rebalancing
rebalance_needs = test_data[test_data['rebalance_flag']].groupby(['product_type_no', 'colour_group_code'])['predicted_inventory_needs'].sum().reset_index()
rebalance_needs['rebalance_units'] = np.round(rebalance_needs['predicted_inventory_needs']).astype(int)

# Summarize the overall predicted inventory needs
inventory_needs = test_data.groupby(['product_type_no', 'colour_group_code'])['predicted_inventory_needs'].sum().reset_index()

# Print results
print("\nPredicted Inventory Needs (Test Period):")
print(inventory_needs)

print("\nPotential Rebalance Needs (Dynamic Low Demand Thresholds):")
print(rebalance_needs)

print("Total Predicted Inventory Needs (Test Period):", int(inventory_needs['predicted_inventory_needs'].sum()))
print("Total Rebalance Needs (Low Demand Adjustments):", int(rebalance_needs['rebalance_units'].sum()))
print("Total Actual Units Sold (Test Period):", int(y_test.sum()))

Training set size: (83854, 12)
Validation set size: (6353, 12)
Test set size: (6594, 12)
Extended Validation MAE: 9.357783724224776
Extended Validation RMSE: 26.614367756322643
Test MAE: 23.652562936002425
Test RMSE: 75.53436267954416

Predicted Inventory Needs (Test Period):
      product_type_no  colour_group_code  predicted_inventory_needs
0                  -1                  6                        291
1                  -1                  7                        173
2                  -1                  9                       1831
3                  -1                 10                        498
4                  -1                 12                        150
...               ...                ...                        ...
1914              515                 12                          2
1915              525                  3                          3
1916              529                 31                         96
1917              529                 52   

In [ ]:
# Ensure last_week_data represents the last week in the test_data
last_week_data = test_data[test_data['week'] == test_data['week'].max()].copy()

# Validate total_units_sold exists and contains no missing values
if 'total_units_sold' not in last_week_data.columns:
    raise KeyError("The column 'total_units_sold' is missing in last_week_data.")

# Handle missing values in total_units_sold, if any
last_week_data['total_units_sold'] = last_week_data['total_units_sold'].fillna(0)

# Initialize the rolling window column
last_week_data['low_demand_4weeks'] = last_week_data['total_units_sold']


         week  total_units_sold  low_demand_4weeks
3  2020-02-24                76                 76
7  2020-02-24                59                 59
11 2020-02-24               451                451
15 2020-02-24               100                100
19 2020-02-24                68                 68


In [ ]:
from datetime import timedelta

# Initialize an empty DataFrame to store predictions for each future week
future_predictions = pd.DataFrame()

# Predict for 4 future weeks
for week_ahead in range(1, 5):
    # Prepare features for prediction using the last week's data
    X_future = last_week_data[feature_columns]  # Ensure feature_columns include all necessary features
    
    # Step 1: Generate predictions from each base model for the current week
    preds_rf = model_rf.predict(X_future)
    preds_lr = model_lr.predict(X_future)
    preds_mlp = model_mlp.predict(X_future)
    
    # Step 2: Create DataFrame with base model predictions to input to the meta-model
    base_preds_df = pd.DataFrame({
        'RandomForest': preds_rf,
        'LinearRegression': preds_lr,
        'MLPRegressor': preds_mlp
    })
    
    # Step 3: Use the meta-model to predict the final units sold
    future_pred = final_model_rf.predict(base_preds_df)
    future_pred = np.round(future_pred).astype(int)  # Ensure integer values
    
    # Step 4: Create a DataFrame to store predictions
    prediction_df = last_week_data[['product_type_no', 'product_type_name', 
                                    'colour_group_code', 'colour_group_name']].copy()
    prediction_df['predicted_units_sold'] = future_pred
    prediction_df['week_ahead'] = week_ahead  # Label the future week
    
    # Step 4b: Add a new 'week' column by advancing by 7 days for each iteration
    if 'week' in last_week_data.columns:
        prediction_df['week'] = last_week_data['week'] + timedelta(weeks=week_ahead)
    else:
        prediction_df['week'] = last_week_data['week'].max() + timedelta(weeks=week_ahead)

    # Step 5: Calculate rolling 4-week cumulative demand
    last_week_data['low_demand_4weeks'] = (
        last_week_data['low_demand_4weeks'].shift(1).fillna(0) + future_pred
    )
    
    # Step 6: Flag for rebalancing
    prediction_df['rebalance_flag'] = np.where(
        (last_week_data['is_peak_season'] & (last_week_data['low_demand_4weeks'] <= last_week_data['low_demand_threshold'])) |
        (~last_week_data['is_peak_season'] & (last_week_data['low_demand_4weeks'] <= (last_week_data['low_demand_threshold'] * 1.5))),
        True,
        False
    )
    
    # Append predictions to the future_predictions DataFrame
    future_predictions = pd.concat([future_predictions, prediction_df], ignore_index=True)
    
    # Step 7: Update last_week_data for the next week's predictions
    # Shift lagged features and update total_units_sold
    last_week_data['lag_units_sold_2weeks'] = last_week_data['lag_units_sold_1week']
    last_week_data['lag_units_sold_1week'] = future_pred
    last_week_data['total_units_sold'] = future_pred
    last_week_data['week'] = last_week_data['week'] + timedelta(weeks=1)  # Move to the next week

# Display the resulting DataFrame with predictions and rebalancing flags for 4 future weeks
print("\nFuture Predictions with Rebalance Flags for 4 Weeks Out:")
print(future_predictions)

# Calculate the total predicted units across all weeks
total_predicted_units = future_predictions['predicted_units_sold'].sum()
print(f"\nTotal Predicted Units for 4 Future Weeks: {total_predicted_units}")



Future Predictions with Rebalance Flags for 4 Weeks Out:
      product_type_no product_type_name  colour_group_code colour_group_name  \
0                  -1           Unknown                  6        Light Grey   
1                  -1           Unknown                  7              Grey   
2                  -1           Unknown                  9             Black   
3                  -1           Unknown                 10             White   
4                  -1           Unknown                 12       Light Beige   
...               ...               ...                ...               ...   
6563              515         Straw hat                 11         Off White   
6564              525          Keychain                  3            Silver   
6565              529       Mobile case                 31      Light Orange   
6566              529       Mobile case                 52              Pink   
6567              532          Umbrella                  9    

In [43]:
# Step 1: Extract the last week of February 2020
data['week'] = pd.to_datetime(data['week']).dt.tz_localize(None)
last_week_data = data[(data['week'] >= "2020-02-24") & (data['week'] <= "2020-02-29")].copy()

# Step 2: Ensure all required columns are present
# If lagged features and rolling columns are not in last_week_data, initialize them as needed
if 'lag_units_sold_1week' not in last_week_data.columns:
    last_week_data['lag_units_sold_1week'] = last_week_data['total_units_sold'].shift(1).fillna(0)
if 'lag_units_sold_2weeks' not in last_week_data.columns:
    last_week_data['lag_units_sold_2weeks'] = last_week_data['total_units_sold'].shift(2).fillna(0)
if 'low_demand_4weeks' not in last_week_data.columns:
    last_week_data['low_demand_4weeks'] = last_week_data['total_units_sold']  # Start with current units sold
if 'is_peak_season' not in last_week_data.columns:
    last_week_data['is_peak_season'] = False  # Assuming False if unknown
if 'low_demand_threshold' not in last_week_data.columns:
    last_week_data['low_demand_threshold'] = last_week_data['total_units_sold'].mean()  # Example threshold

# Initialize an empty DataFrame to store predictions for each future week
future_predictions = pd.DataFrame()

# Step 3: Predict for 4 future weeks
for week_ahead in range(1, 5):
    # Prepare features for prediction using the last week's data
    X_future = last_week_data[feature_columns]  # Ensure feature_columns include all necessary features
    
    # Step 1: Generate predictions from each base model for the current week
    preds_rf = model_rf.predict(X_future)
    preds_lr = model_lr.predict(X_future)
    preds_mlp = model_mlp.predict(X_future)
    
    # Step 2: Create DataFrame with base model predictions to input to the meta-model
    base_preds_df = pd.DataFrame({
        'RandomForest': preds_rf,
        'LinearRegression': preds_lr,
        'MLPRegressor': preds_mlp
    })
    
    # Step 3: Use the meta-model to predict the final units sold
    future_pred = final_model_rf.predict(base_preds_df)
    future_pred = np.round(future_pred).astype(int)  # Ensure integer values
    
    # Step 4: Create a DataFrame to store predictions for this week
    prediction_df = last_week_data[['product_type_no', 'product_type_name', 
                                    'colour_group_code', 'colour_group_name']].copy()
    prediction_df['predicted_units_sold'] = future_pred
    prediction_df['week_ahead'] = week_ahead  # Label the future week
    
    # Step 5: Update the rolling 4-week cumulative demand
    last_week_data['low_demand_4weeks'] = (
        last_week_data['low_demand_4weeks'].shift(1).fillna(0) + future_pred
    )
    
    # Step 6: Flag items for rebalancing based on the updated demand threshold
    prediction_df['rebalance_flag'] = np.where(
        (last_week_data['is_peak_season'] & (last_week_data['low_demand_4weeks'] <= last_week_data['low_demand_threshold'])) |
        (~last_week_data['is_peak_season'] & (last_week_data['low_demand_4weeks'] <= (last_week_data['low_demand_threshold'] * 1.5))),
        True,
        False
    )
    
    # Append predictions to the future_predictions DataFrame
    future_predictions = pd.concat([future_predictions, prediction_df], ignore_index=True)
    
    # Step 7: Update last_week_data to simulate moving into the next week
    # Shift lagged features to reflect the new week's demand
    last_week_data['lag_units_sold_2weeks'] = last_week_data['lag_units_sold_1week']
    last_week_data['lag_units_sold_1week'] = future_pred
    last_week_data['total_units_sold'] = future_pred  # Update with predicted units for the new week

# Step 8: Display the resulting DataFrame with predictions and rebalancing flags for 4 future weeks
print("\nFuture Predictions with Rebalance Flags for 4 Weeks Out:")
print(future_predictions)

# Calculate the total predicted units across all weeks
total_predicted_units = future_predictions['predicted_units_sold'].sum()
print(f"\nTotal Predicted Units for 4 Future Weeks: {total_predicted_units}")



Future Predictions with Rebalance Flags for 4 Weeks Out:
      product_type_no product_type_name  colour_group_code colour_group_name  \
0                  -1           Unknown                  6        Light Grey   
1                  -1           Unknown                  7              Grey   
2                  -1           Unknown                  9             Black   
3                  -1           Unknown                 10             White   
4                  -1           Unknown                 12       Light Beige   
...               ...               ...                ...               ...   
6563              515         Straw hat                 11         Off White   
6564              525          Keychain                  3            Silver   
6565              529       Mobile case                 31      Light Orange   
6566              529       Mobile case                 52              Pink   
6567              532          Umbrella                  9    

In [46]:
# Set up the schema if needed and push inventory_needs to PostgreSQL
with engine.connect() as conn:
    conn.execute(text(f'SET search_path TO {my_schema};'))
    future_predictions.to_sql('future_predictions', con=engine, schema=my_schema, if_exists='replace', index=False)

print("Table successfully pushed to the database!")


Table successfully pushed to the database!
